In [1]:
import matplotlib.pyplot as plt
import scipy

import torch
from torch.utils.data import DataLoader

from transformers import AutoTokenizer, LlamaForCausalLM, LlamaConfig
from datasets import load_dataset
import evaluate

import numpy as np
import pandas as pd
from tqdm import tqdm

import os
import argparse
import json
import hashlib
import subprocess

/nlp/scr/rohithk/miniconda3/envs/blackbox-model-tracing/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def eval_tiny(model_path, eval_texts):
    perplexity = evaluate.load("perplexity", module_type="metric")
    result = perplexity.compute(model_id=model_path,
                                add_start_token=True,
                                predictions=eval_texts)
    pplx = np.log(result['perplexities'])

    return pplx

In [9]:
model_id = 1
control_id = 0
model_epoch = 9

synth_id = 0
synth_epoch = 0
SYNTH_HASH = "d50a185b" # "63f5217d"

MODEL_HASH = 'e4c9c1ac'
df_path = f'./kgram-analysis/models/tiny_ref_model_{MODEL_HASH}/tinystories.csv'
model_path = f'./kgram-analysis/models/tiny_ref_model_{MODEL_HASH}/epoch-{model_epoch}-index-{model_id}'
control_path = f'./kgram-analysis/models/tiny_ref_model_{MODEL_HASH}/epoch-{model_epoch}-index-{control_id}'

synth_path = f'./synthetic_references/debug/tiny_synth_model_{SYNTH_HASH}/epoch-{synth_epoch}-index-{synth_id}'

N_TRAIN_SAMPLES = 10000

In [6]:
print(synth_path)

./synthetic_references/debug/tiny_synth_model_63f5217d/epoch-1-index-0


In [4]:
dataset = load_dataset("roneneldan/TinyStories")

texts = dataset["train"]["text"][:N_TRAIN_SAMPLES]
texts = [item for item in texts if item != ""]

In [10]:
order_epoch = 0

# model_pplx = eval_tiny(model_path,texts)
# ctrl_pplx = eval_tiny(control_path,texts)
synth_pplx = eval_tiny(synth_path,texts)

df = pd.read_csv(df_path)

stat = scipy.stats.spearmanr(np.argsort(df[f'order-{model_id}-epoch-{order_epoch}']), synth_pplx-ctrl_pplx)
og_stat = scipy.stats.spearmanr(np.argsort(df[f'order-{model_id}-epoch-{order_epoch}']), model_pplx-ctrl_pplx)


print(stat)
print(og_stat)

100%|████████████████████████████████████████████████████████████████| 625/625 [01:12<00:00,  8.61it/s]


SignificanceResult(statistic=-0.030245459942454603, pvalue=0.002487483471811234)
SignificanceResult(statistic=-0.36053128584531285, pvalue=1.107603753391792e-304)
